In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import time

torch.set_printoptions(edgeitems=2)
# 默认当前数据处理
torch.manual_seed(123)

In [2]:
# 加载数据集
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

data_path = '../data-unversioned/p1ch7/'

trainset = torchvision.datasets.CIFAR10(root=data_path, train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=data_path, train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# 定义模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
moedel = Net()
numel_list = [p.numel() for p in moedel.parameters()]
sum(numel_list), numel_list

(62006, [450, 6, 2400, 16, 48000, 120, 10080, 84, 840, 10])

In [5]:
# import torch.multiprocessing as mp
#
# # 设置MPS的最大内存限制为4GB
# if mp.get_start_method() != 'spawn':
#     mp.set_start_method('spawn')
# mp.set_sharing_strategy('file_system')
# mp.multiprocessing.set_start_method("spawn", force=True)
# mp.get_context("spawn").mp_metrics._max_dev_mem_size = 4294967296

In [6]:
net = Net()
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

start_time = time.time()
# 训练模型
for epoch in range(100):  # 多次循环遍历数据集
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # 每2000个小批量打印一次损失值
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

end_time = time.time()
total_time = end_time - start_time
print("运行时间为：", total_time, "秒")
print('Finished Training')

[1,  2000] loss: 2.216
[1,  4000] loss: 1.874
[1,  6000] loss: 1.675
[1,  8000] loss: 1.606
[1, 10000] loss: 1.548
[1, 12000] loss: 1.490
[2,  2000] loss: 1.417
[2,  4000] loss: 1.392
[2,  6000] loss: 1.368
[2,  8000] loss: 1.346
[2, 10000] loss: 1.323
[2, 12000] loss: 1.307
[3,  2000] loss: 1.233
[3,  4000] loss: 1.225
[3,  6000] loss: 1.235
[3,  8000] loss: 1.186
[3, 10000] loss: 1.209
[3, 12000] loss: 1.178
[4,  2000] loss: 1.092
[4,  4000] loss: 1.122
[4,  6000] loss: 1.127
[4,  8000] loss: 1.103
[4, 10000] loss: 1.097
[4, 12000] loss: 1.081
[5,  2000] loss: 0.997
[5,  4000] loss: 1.001
[5,  6000] loss: 1.034
[5,  8000] loss: 1.036
[5, 10000] loss: 1.021
[5, 12000] loss: 1.041
[6,  2000] loss: 0.935
[6,  4000] loss: 0.952
[6,  6000] loss: 0.957
[6,  8000] loss: 0.984
[6, 10000] loss: 0.963
[6, 12000] loss: 0.968
[7,  2000] loss: 0.856
[7,  4000] loss: 0.911
[7,  6000] loss: 0.927
[7,  8000] loss: 0.919
[7, 10000] loss: 0.947
[7, 12000] loss: 0.921
[8,  2000] loss: 0.817
[8,  4000] 

In [7]:
# device = torch.device('cuda')
# device = torch.device('mps')
net = Net()
# net = net.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

start_time = time.time()
# 训练模型
for epoch in range(2):  # 多次循环遍历数据集
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        # outputs = net(inputs.to(device))
        outputs = net(inputs)
        # loss = criterion(outputs, labels.to(device))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # 每2000个小批量打印一次损失值
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

end_time = time.time()
total_time = end_time - start_time
print("运行时间为：", total_time, "秒")
print('Finished Training')

[1,  2000] loss: 2.264
[1,  4000] loss: 1.922
[1,  6000] loss: 1.671
[1,  8000] loss: 1.612
[1, 10000] loss: 1.534
[1, 12000] loss: 1.492
[2,  2000] loss: 1.406
[2,  4000] loss: 1.385
[2,  6000] loss: 1.348
[2,  8000] loss: 1.333
[2, 10000] loss: 1.311
[2, 12000] loss: 1.266
运行时间为： 131.9481692314148 秒
Finished Training


In [9]:
# 测试模型
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

RuntimeError: slow_conv2d_forward_mps: input(device='cpu') and weight(device=mps:0')  must be on the same device